# Entrenament del model

In [94]:
import pandas as pd
import joblib, os, shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime

In [ ]:
data = pd.read_csv("../res/dataset_final.csv")
data = data.set_index("_id")
# No se si es bona idea del tot igual mos fiquen pegues
# Si continuhem eliminant esta marca de manera manual deuriem de explicar molt be el perque
# Hem de comprobar si mos dona problemes despres de els outliders
data = data[data["marca"].str.lower() != "thuraya"]
data

,ano,almacenamiento,marca,pantalla_in,pantalla_tipo,velocidad_cpu_ghz,ram,grosor,peso,ancho_px,alto_px,bateria,promedio_valoraciones,precio_anterior,precio_actual
_id,,,,,,,,,,,,,,,
B00IRZ8EQC,2013.0,256.0,samsung,4.30,lcd,1.7,1.0,1.27,77.11,540.0,960.0,5000.0,279.459459,199.99,245.00
B00J8OA220,2024.0,160.0,inmarsat,2.00,lcd,2.9,2.0,10.16,318.00,240.0,320.0,160.0,0.800000,199.99,933.00
B00JC8MD7Y,2024.0,256.0,samsung,6.70,superamoled,2.2,6.0,0.77,179.00,1080.0,2400.0,4500.0,1761.818182,279.99,279.99
B00TKALUDC,2015.0,128.0,ttfone,2.00,lcd,0.0,4.0,1.80,75.00,1080.0,2400.0,800.0,168.571429,45.99,41.99
B00TUXHZTW,2015.0,128.0,doro,2.00,lcd,1.2,12.0,1.90,74.00,3840.0,2160.0,800.0,74.000000,48.67,48.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,2024.0,128.0,honor,6.56,lcd,22.1,4.0,2.57,254.42,1920.0,1080.0,5000.0,1.702128,199.99,99.90
B09QH71RZR,2024.0,128.0,xiaomi,6.43,amoled,2.0,6.0,0.81,179.00,1920.0,1080.0,5000.0,3821.818182,165.39,127.00
B0DK4NBZWB,2024.0,256.0,oppo,6.67,amoled,3.4,8.0,2.57,254.42,1080.0,2400.0,5000.0,2.291667,199.99,299.00


In [81]:
data = data.dropna()

## Transformació de valors

### Columnes de text

In [82]:
def aplicar_labelencoder(df, columnas_object=None):
    if not columnas_object:
        columnas_object = df.select_dtypes(include=['object']).columns
    df_encoded = df[columnas_object]
    for columna in columnas_object:
        encoder = LabelEncoder()
        df_encoded[columna] = encoder.fit_transform(df_encoded[columna])
    return df_encoded

# Ejemplo de uso
df_label = aplicar_labelencoder(data)

/tmp/ipykernel_12487/223279276.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded[columna] = encoder.fit_transform(df_encoded[columna])
/tmp/ipykernel_12487/223279276.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded[columna] = encoder.fit_transform(df_encoded[columna])


In [ ]:
def aplicar_onehotencoder(df, columnas_object = None):
    if not columnas_object: columnas_object = df.select_dtypes(include=['object']).columns
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    encoded_data = encoder.fit_transform(df[columnas_object])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columnas_object))
    encoded_df.index = df.index
    return encoded_df, encoder

df_onehot, encoder_ohe = aplicar_onehotencoder(data)
df_onehot

,marca_8849pro,marca_akozon,marca_alcatel,marca_apple,marca_artfone,marca_asus,marca_azuiqimeng,marca_bewinner,marca_blackview,marca_ciciglow,...,pantalla_tipo_dlp,pantalla_tipo_fhd+,pantalla_tipo_hd,pantalla_tipo_hd+,pantalla_tipo_ips,pantalla_tipo_lcd,pantalla_tipo_led,pantalla_tipo_oled,pantalla_tipo_superamoled,pantalla_tipo_xdr
_id,,,,,,,,,,,,,,,,,,,,,
B00IRZ8EQC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
B00J8OA220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
B00JC8MD7Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
B00TKALUDC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
B00TUXHZTW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
B09QH71RZR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0DK4NBZWB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Columnes numeriques

In [84]:
def aplicar_standardscaler(df, columnas_numericas=None):
    if columnas_numericas is None:
        columnas_numericas = df.select_dtypes(include=['number']).columns
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[columnas_numericas])
    scaled_df = pd.DataFrame(scaled_data, columns=columnas_numericas, index=df.index)
    return scaled_df, scaler

df_scaled, scaler = aplicar_standardscaler(data)
print(data.select_dtypes(include=['number']).columns)

Index(['ano', 'almacenamiento', 'pantalla_in', 'velocidad_cpu_ghz', 'ram',
       'grosor', 'peso', 'ancho_px', 'alto_px', 'bateria',
       'promedio_valoraciones', 'precio_anterior', 'precio_actual'],
      dtype='object')


### Unim les columnes

In [85]:
df_onehot.index = df_scaled.index
df_normalitzat = pd.concat([df_onehot, df_scaled], axis=1)
df_normalitzat

,marca_8849pro,marca_akozon,marca_alcatel,marca_apple,marca_artfone,marca_asus,marca_azuiqimeng,marca_bewinner,marca_blackview,marca_ciciglow,...,velocidad_cpu_ghz,ram,grosor,peso,ancho_px,alto_px,bateria,promedio_valoraciones,precio_anterior,precio_actual
_id,,,,,,,,,,,,,,,,,,,,,
B00IRZ8EQC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.087811,-0.988617,-0.307234,-1.825096,-1.304233,-1.302408,0.030563,0.152143,-0.319815,0.084754
B00J8OA220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.069570,-0.875917,1.865245,0.646414,-1.842790,-2.223395,-1.787482,-0.282856,-0.319815,3.171401
B00JC8MD7Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.080211,-0.425116,-0.429420,-0.779714,-0.334832,0.769812,-0.157252,2.466166,0.024649,0.241733
B00TKALUDC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.113653,-0.650517,-0.177716,-1.846744,-0.334832,0.769812,-1.547080,-0.020958,-0.982909,-0.826031
B00TUXHZTW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.095412,0.251085,-0.153279,-1.857004,4.619886,0.424442,-1.547080,-0.168588,-0.971370,-0.796062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.222290,-0.650517,0.010452,-0.005911,1.173125,-1.129723,0.030563,-0.281448,-0.319815,-0.566223
B09QH71RZR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.083251,-0.425116,-0.419645,-0.779714,1.173125,-1.129723,0.030563,5.681911,-0.468796,-0.444642
B0DK4NBZWB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.061970,-0.199716,0.010452,-0.005911,-0.334832,0.769812,0.030563,-0.280528,-0.319815,0.327020


In [86]:
# Separar features y target
X = df_normalitzat.drop(columns=["precio_actual"])
y = df_normalitzat["precio_actual"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## GRADIENT BOSSTING REGRESSOR

In [87]:
# Entrenar modelo GradientBoostingRegressor
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
history_gb = gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)


# Entrenar modelo RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
history_rf = rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)


## EVALUACION

In [88]:
# Andreu: Son les metriques que hem de gastar o i han millors per a este cas de us?
# Evaluar modelo GradientBoostingRegressor
gb_results = {
    "MAE": mean_absolute_error(y_test, y_pred_gb),
    "MSE": mean_squared_error(y_test, y_pred_gb),
    "R2 Score": r2_score(y_test, y_pred_gb),
}

# Evaluar modelo RandomForestRegressor
rf_results = {
    "MAE": mean_absolute_error(y_test, y_pred_rf),
    "MSE": mean_squared_error(y_test, y_pred_rf),
    "R2 Score": r2_score(y_test, y_pred_rf),
}

# Mostrar resultados
print("GradientBoostingRegressor:")
for metric, value in gb_results.items():
    print(f"  {metric}: {value}")

print("\nRandomForestRegressor:")
for metric, value in rf_results.items():
    print(f"  {metric}: {value}")

GradientBoostingRegressor:
  MAE: 0.24241679902377533
  MSE: 0.20542094029238905
  R2 Score: 0.761507034045057

RandomForestRegressor:
  MAE: 0.17830621708705643
  MSE: 0.13748593200201698
  R2 Score: 0.8403793320507185


## GUARDAR MODEL

In [89]:
fecha_hora_actual = datetime.now().strftime("%d_%m_%Y_%H_%M")
model_type = "gb"

folder_name = f"model_{model_type}_{fecha_hora_actual}"
models_path = os.path.join("./models", folder_name)
os.makedirs(models_path, exist_ok=True)

joblib.dump(gb_model, os.path.join(models_path, f'model.joblib'))
joblib.dump(encoder_ohe, os.path.join(models_path, 'encoder_ohe.joblib'))
joblib.dump(scaler, os.path.join(models_path, 'scaler.joblib'))

X_train.to_csv(os.path.join(models_path, f'X_train.csv'))
X_test.to_csv(os.path.join(models_path, f'X_test.csv'))
y_train.to_csv(os.path.join(models_path, f'y_train.csv'))
y_test.to_csv(os.path.join(models_path, f'y_test.csv'))

try:
    shutil.make_archive(models_path, 'zip', root_dir=models_path)
    shutil.rmtree(models_path)
    print(f"Carpeta comprimida exitosamente a {folder_name}.zip")
except Exception as e:
    print(f"Error: {e}")

print(f"Modelo y transformadores guardados en: {models_path}")

Carpeta comprimida exitosamente a model_gb_23_02_2025_18_01.zip
Modelo y transformadores guardados en: ./models/model_gb_23_02_2025_18_01


## Prediccions

In [90]:
peticion = {
    "ano":2025,
    "almacenamiento": 100,
    "marca": 'apple',
    "pantalla_in": 6.7, 
    "pantalla_tipo": 'lcd', 
    "velocidad_cpu_ghz": 6.2,
    "ram": 32, 
    "grosor": 0.77, 
    "peso":300,
    'ancho_px':1080, 
    'alto_px':2400,     
    "bateria": 5000,  
    'promedio_valoraciones':13566.95, 
    'precio_anterior':500,
    'precio_actual': 0
}

peticion_df = pd.DataFrame([peticion])
scaler = joblib.load('./models/model_gb_23_02_2025_13_50/scaler.joblib')
encoder = joblib.load('./models/model_gb_23_02_2025_13_50/encoder_ohe.joblib')
model = joblib.load('./models/model_gb_23_02_2025_13_50/model.joblib')

peticion_scaled = scaler.transform(peticion_df.select_dtypes(include=['number']))
peticion_encoder = encoder.transform(peticion_df.select_dtypes(include=['object']))

peticion_df_scaled = pd.DataFrame(peticion_scaled, columns=peticion_df.select_dtypes(include=['number']).columns)
peticion_df_encoded = pd.DataFrame(peticion_encoder, columns=encoder.get_feature_names_out())

peticion_final = pd.concat([peticion_df_scaled, peticion_df_encoded], axis=1)
peticion_final.drop(['precio_actual'], inplace = True, axis = 1)

X_train = pd.read_csv("./models/model_gb_23_02_2025_13_50/X_train.csv")
X_train.drop(['_id'], inplace = True, axis = 1)
peticion_final = peticion_final[X_train.columns]

prediccio = model.predict(peticion_final)

peticion_scaled_with_pred = peticion_scaled.copy()
peticion_scaled_with_pred[:, peticion_df_scaled.columns.get_loc('precio_actual')] = prediccio
prediccion_escalada_inversa = scaler.inverse_transform(peticion_scaled_with_pred)

# Extract the inverse-transformed prediction
prediccion_final = prediccion_escalada_inversa[0, peticion_df_scaled.columns.get_loc('precio_actual')]

print(prediccion_final)

210.01224999050226
